In [393]:
!pip install tf-models-official==2.5.0

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
import tensorflow_text as tf_text
from official.nlp.transformer.utils import tokenizer as tok

In [4]:
from tensorflow.keras.layers.experimental import preprocessing
from official.nlp.transformer import metrics, embedding_layer


In [5]:
from official.nlp.transformer import transformer_main, model_params
from official.utils.misc import distribution_utils
from official.nlp.transformer import transformer
from official.nlp.transformer import optimizer




In [6]:
%load_ext tensorboard

In [7]:
path_data = "data/"
path_model = "../../../tensorboard_data/"

In [8]:
COL_TYPES =  [0]+ [0] + ['a']+['a']
DEFAULTS = [tf.int64, tf.int64, tf.int64 ,tf.int64]
COL_NAMES = ["DocID","LineID","RawText","target"]

In [8]:
#AGE_CUT_VALUES = np.concatenate([[-2, -1, 1], (5 + np.arange(0, 125, 5))])

### Pre-processing pipeline

Faire attention à ne pas enlever toutes les points d'exclamationsqui peuvent représenter des mots 

In [9]:
def tf_lower_and_split_punct(text):
  # Split accecented characters.
  #import unidecode
  #text = unidecode.unidecode(text)
  text = tf.strings.lower(text)
  # Keep space, a to z, and select punctuation.
  text = tf.strings.regex_replace(text, '^ a-z.!?,¿', '')
  # Add spaces around punctuation.
  text = tf.strings.regex_replace(text, '.?!,¿', r' \0 ')
  # Strip whitespace.
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END] '], separator=' ')
  return text




In [22]:
input_text_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=None)

target_processor = preprocessing.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=None)



In [11]:
# input_text_processor.adapt(dataset)
# take very long time, different way for the moment
data = pd.read_csv("data/corpus_train_code_target.csv", encoding='utf-8', sep=";")
input_text_processor.adapt(data.RawText)
target_processor.adapt(data.target)



In [23]:


input_text_processor.adapt(df.ADAPT)




In [26]:
import pickle

pickle.dump({'config': input_text_processor.get_config(),
             'weights': input_text_processor.get_weights()}
            , open("full_adapt_processor.pkl", "wb"))

In [13]:
import pickle

pickle.dump({'config': input_text_processor.get_config(),
             'weights': input_text_processor.get_weights()}
            , open("full_adapt_processor.pkl", "wb"))
pickle.dump({'config': target_processor.get_config(),
             'weights': target_processor.get_weights()}
            , open("full_target_processor.pkl", "wb"))
    


In [24]:
len(input_text_processor.get_vocabulary())

136158

In [16]:
len(target_processor.get_vocabulary())

20861

In [25]:
input_text_processor.get_vocabulary()

['',
 '[UNK]',
 '[START]',
 '[END]',
 '[sepcode]',
 'arret',
 'insuffisance',
 'cardio-respiratoire',
 'cardiaque',
 'respiratoire',
 'cancer',
 'pneumopathie',
 'choc',
 'pulmonaire',
 'aigue',
 'defaillance',
 'metastases',
 'renale',
 'syndrome',
 'etat',
 'terminale',
 'detresse',
 'avc',
 'cardiopathie',
 'demence',
 'severe',
 'ischemique',
 'septique',
 'chronique',
 'decompensation',
 'maladie',
 'multiviscerale',
 'alzheimer',
 'hta',
 'neoplasie',
 'metastatique',
 '!',
 'coma',
 'carcinome',
 'inhalation',
 'mort',
 'adenocarcinome',
 'bronchique',
 'troubles',
 'hemorragie',
 'infarctus',
 'denutrition',
 'metastase',
 'diabete',
 'alteration',
 'general',
 'cerebral',
 'hepatique',
 'acr',
 'infection',
 'stade',
 'vasculaire',
 'gauche',
 'sepsis',
 '|',
 'cirrhose',
 'cachexie',
 'embolie',
 'probable',
 'sein',
 'neo',
 'bpco',
 'grabataire',
 'accident',
 'evolution',
 'hemorragique',
 'glissement',
 'myocarde',
 'de',
 'hepatiques',
 'tumeur',
 'cerebrale',
 'digestiv

In [25]:
def preprocess(features):


    txt = input_text_processor(features.pop('RawText'))
    
    labels = target_processor(features.pop('target'))
    
    return (txt,labels), labels



In [26]:
def input_fn(dataset_name, batch_size, preprocess = preprocess):

    data = tf.data.experimental.make_csv_dataset(
        path_data+'%s.csv' % dataset_name,
        batch_size=batch_size,
        column_names=COL_NAMES,
        column_defaults=COL_TYPES,
        header=True,
        shuffle=True,
        shuffle_buffer_size=20000,
        sloppy=True,
        prefetch_buffer_size=1,
        use_quote_delim=True,
        field_delim=';'

    )

    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.prefetch(buffer_size=1)

    return data


## Transformer

In [16]:
train_ds = input_fn(dataset_name = "extrait500_corpus_train_code_target",batch_size =32, preprocess  = preprocess)
eval_ds = input_fn(dataset_name = "extrait500_corpus_val_code_target",batch_size =32, preprocess  = preprocess)


train_ds

<PrefetchDataset element_spec=((TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None)), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [17]:
def create_optimizer(params):
    lr_schedule = optimizer.LearningRateSchedule(
        params['learning_rate'], params['hidden_size'],
        params['learning_rate_warmup_steps']
    )

    opt = tf.keras.optimizers.Adam(
        lr_schedule,
        params['optimizer_adam_beta1'],
        params["optimizer_adam_beta2"],
        epsilon=params["optimizer_adam_epsilon"]
    )

    return opt

In [18]:
def train_model(train_ds,eval_ds,params, model_dir):
       
    model = transformer.create_model(params, is_train=True)
    opt = create_optimizer(params)

    checkpoint = tf.train.Checkpoint(model=model, optimizer=opt)
    latest_checkpoint = tf.train.latest_checkpoint(model_dir)
    if latest_checkpoint:
        checkpoint.restore(latest_checkpoint)

    board = tf.keras.callbacks.TensorBoard(
        log_dir=model_dir, write_graph=False, update_freq=500
    )
    model.compile(opt)

    model.fit(
        x = train_ds,
        epochs=5,
        steps_per_epoch=10000,
        verbose=1,
        validation_data=eval_ds,
        validation_steps=100,
        callbacks=[board])
    re




In [20]:
params = model_params.TINY_PARAMS
params["batch_size"] = params["default_batch_size"] = 16
params["use_synthetic_data"] = True
params["hidden_size"] = 12
params["num_hidden_layers"] = 1
params["filter_size"] = 14
params["num_heads"] = 3
params["extra_decode_length"] = 2
params["beam_size"] = 3
params["dtype"] = tf.float32
params['input_vocab_size'] = len(input_text_processor.get_vocabulary())
params['vocab_size'] = len(target_processor.get_vocabulary())
params['default_batch_size'] = 80*3




In [21]:

params


defaultdict(<function official.nlp.transformer.model_params.<lambda>()>,
            {'default_batch_size': 240,
             'default_batch_size_tpu': 1024,
             'max_length': 256,
             'initializer_gain': 1.0,
             'vocab_size': 34,
             'hidden_size': 12,
             'num_hidden_layers': 1,
             'num_heads': 3,
             'filter_size': 14,
             'layer_postprocess_dropout': 0.1,
             'attention_dropout': 0.1,
             'relu_dropout': 0.1,
             'label_smoothing': 0.1,
             'learning_rate': 2.0,
             'learning_rate_decay_rate': 1.0,
             'learning_rate_warmup_steps': 16000,
             'optimizer_adam_beta1': 0.9,
             'optimizer_adam_beta2': 0.997,
             'optimizer_adam_epsilon': 1e-09,
             'extra_decode_length': 2,
             'beam_size': 3,
             'alpha': 0.6,
             'use_tpu': False,
             'static_batch': False,
             'allow_ffn_pad':

In [21]:

drop_out_val = 0.3
params['layer_postprocess_dropout'] = drop_out_val
params['attention_dropout'] = drop_out_val
params['relu_dropout'] = drop_out_val
model_dir = path_model

model = transformer.create_model(params, is_train=True)
opt = create_optimizer(params)
model.compile(opt)
board = tf.keras.callbacks.TensorBoard(
    log_dir=model_dir, write_graph=True, update_freq=500
)
model.fit(
    x = train_ds,
    epochs=5,
    steps_per_epoch=10000,
    verbose=1,
    validation_data=eval_ds,
    validation_steps=100,
    callbacks=[board]
)


Instructions for updating:
DenseEinsum is deprecated. Please use tf.keras.experimental.EinsumDense layer instead.
Epoch 1/5


KeyboardInterrupt: 

Le modèle prédit en se servant des logits

# Validation

In [584]:
dir(model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_captured_weight_regularizer',
 '_cast_single_i

In [673]:
model.save_weights("Model_poids/")



In [22]:
Model_test = transformer.create_model(params, is_train=False)
Model_test.load_weights("Model_poids/")

Instructions for updating:
DenseEinsum is deprecated. Please use tf.keras.experimental.EinsumDense layer instead.


In [27]:
dataset = input_fn(dataset_name = "extrait500_corpus_train_code_target",batch_size =32, preprocess  = preprocess
                  )
iterator = iter(dataset)
iterator

In [28]:
data = pd.read_csv("data/extrait500_corpus_test_code_target.csv", encoding='utf-8', sep=";")
len(data)

748

In [29]:
a = iterator.get_next()
a[0][1]

<tf.Tensor: shape=(32, 5), dtype=int64, numpy=
array([[ 2, 11,  3,  0,  0],
       [ 2, 27,  3,  0,  0],
       [ 2,  9, 25,  3,  0],
       [ 2,  7,  8, 28,  3],
       [ 2, 21, 20,  3,  0],
       [ 2,  4, 17,  3,  0],
       [ 2, 31, 30,  3,  0],
       [ 2,  4, 15,  3,  0],
       [ 2,  5, 10,  3,  0],
       [ 2,  4, 17,  3,  0],
       [ 2,  9, 12,  3,  0],
       [ 2, 11,  3,  0,  0],
       [ 2,  7,  6, 13,  3],
       [ 2,  4, 15,  3,  0],
       [ 2, 24,  3,  0,  0],
       [ 2, 16,  3,  0,  0],
       [ 2,  7,  6,  3,  0],
       [ 2, 11, 18,  3,  0],
       [ 2,  5, 10,  3,  0],
       [ 2,  9, 12,  3,  0],
       [ 2,  7,  8,  3,  0],
       [ 2, 11,  3,  0,  0],
       [ 2, 33, 32,  3,  0],
       [ 2, 11,  3,  0,  0],
       [ 2, 23, 22,  3,  0],
       [ 2,  7,  8,  3,  0],
       [ 2, 23, 22,  3,  0],
       [ 2,  9, 12,  3,  0],
       [ 2,  7,  6,  3,  0],
       [ 2, 31, 30,  3,  0],
       [ 2, 16,  3,  0,  0],
       [ 2,  4, 17,  3,  0]], dtype=int64)>

In [30]:

for i in range(len(a[0][0])):
    vocab = input_text_processor.get_vocabulary()
    print(" ".join([vocab[each] for each in tf.squeeze(a[0][0][i])])
)

[START] pneumopathie [END]  
[START] cachexie [END]  
[START] defaillance polyviscerale [END] 
[START] insuffisance cardiaque terminale [END]
[START] cardiopathie ischemique [END] 
[START] arret cardiorespiratoire [END] 
[START] embolie pulmonaire [END] 
[START] arret cardio-respiratoire [END] 
[START] choc septique [END] 
[START] arret cardiorespiratoire [END] 
[START] defaillance multiviscerale [END] 
[START] pneumopathie [END]  
[START] insuffisance respiratoire aigu [END]
[START] arret cardio respiratoire [END]
[START] hta [END]  
[START] survenue d&aposun coma [END]
[START] insuffisance respiratoire [END] 
[START] pneumopathie d&aposinhalation [END] 
[START] choc septique [END] 
[START] defaillance multiviscerale [END] 
[START] insuffisance cardiaque [END] 
[START] pneumopathie [END]  
[START] carcinose peritoneale [END] 
[START] pneumopathie [END]  
[START] infarctus du myocarde [END]
[START] insuffisance cardiaque [END] 
[START] infarctus du myocarde [END]
[START] defaillance mu

In [33]:
test = Model_test.predict(a[0][0])
test

1/1 [==============================] - 2s 2s/step


[array([[ 2, 11,  3,  3,  3,  3,  3],
        [ 2, 27,  3,  3,  3,  3,  3],
        [ 2,  9, 12,  3,  3,  3,  3],
        [ 2,  7,  8,  3,  3,  3,  3],
        [ 2, 21, 20,  3,  3,  3,  3],
        [ 2,  4, 17,  3,  3,  3,  3],
        [ 2, 31, 30,  3,  3,  3,  3],
        [ 2,  4, 15,  3,  3,  3,  3],
        [ 2,  5, 10,  3,  3,  3,  3],
        [ 2,  4, 17,  3,  3,  3,  3],
        [ 2,  9, 12,  3,  3,  3,  3],
        [ 2, 11,  3,  3,  3,  3,  3],
        [ 2,  7,  6,  3,  3,  3,  3],
        [ 2,  4, 17,  3,  3,  3,  3],
        [ 2, 24,  3,  3,  3,  3,  3],
        [ 2, 16,  3,  3,  3,  3,  3],
        [ 2,  7,  6,  3,  3,  3,  3],
        [ 2, 11, 18,  3,  3,  3,  3],
        [ 2,  5, 10,  3,  3,  3,  3],
        [ 2,  9, 12,  3,  3,  3,  3],
        [ 2,  7,  8,  3,  3,  3,  3],
        [ 2, 11,  3,  3,  3,  3,  3],
        [ 2, 33, 32,  3,  3,  3,  3],
        [ 2, 11,  3,  3,  3,  3,  3],
        [ 2, 23, 22,  3,  3,  3,  3],
        [ 2,  7,  8,  3,  3,  3,  3],
        [ 2,

In [ ]:
np.shape(test[0])

In [ ]:
for i in range(len(test[0])):
    vocab = target_processor.get_vocabulary()
    print(" ".join([vocab[each] for each in tf.squeeze(test[0][i])])
)

In [ ]:
for i in range(len(a[1])):
    vocab = target_processor.get_vocabulary()
    print(" ".join([vocab[each] for each in tf.squeeze(a[1][i])])
)

In [ ]:
from sklearn.metrics import accuracy_score

def precision(prediction, labels):
    labels = labels.numpy()
    prediction = np.where(prediction==2,0,prediction)
    prediction = np.where(prediction==3,0,prediction)
    labels = np.where(labels==2,0,labels)
    labels = np.where(labels==3,0,labels)
    size = np.shape(labels)[1]
    if np.shape(prediction)[1]<size: #On ajoute des zeros si la prediction n'est pas assez longue pour correspondre aux labels
        result = np.zeros(labels.shape)
        result[:prediction.shape[0],:prediction.shape[1]] = prediction
        prediction = result
    y_pred = prediction[:,:size] #On réduit la prédiction à la taille des labels sinon, et on exclut le [START]
    precision = 0
    for i in range(labels.shape[0]):
        k = 1
        for j in range(size):
            if labels[i,k] != 0 :
                k+=1
        print("La précision de la phrase", i+1,"est de :", accuracy_score(labels[i,1:k], y_pred[i,1:k]))
        precision += accuracy_score(labels[i,1:k], y_pred[i,1:k])
    precision_moyenne = precision/labels.shape[0]
    print("La précision moyenne du modèle sur ce batch est de : ", precision_moyenne)
    return
    

In [ ]:
precision(test[0],a[1])

In [31]:
import tensorflow as tf
import tensorflow.keras.backend as K
from official.nlp.transformer import metrics as t_metrics


def isin(padded_outputs,padded_labels):
    """Percentage of time logits contains labels on non-0s."""
    with tf.name_scope("isin"):
        tile_multiple = tf.shape(padded_labels)[-1]
        tiled_outputs = tf.tile(tf.expand_dims(padded_outputs, axis=-1), multiples=[1, 1, tile_multiple])
        tiled_labels = tf.reshape(tf.tile(padded_labels, multiples=[1, tile_multiple]), [-1, tile_multiple, tile_multiple])
        equal = tf.equal(tiled_outputs, tiled_labels)
        any = tf.reduce_any(equal, axis=-1)
        return any


def true_false_positives(logits, labels):
    size = np.shape(labels)[1]
    logits = logits[:,:size]
    labels = labels[:,:size]
    logits = np.where(logits==2,0,logits)
    logits = np.where(logits==3,0,logits)
    labels = np.where(labels==2,0,labels)
    labels = np.where(labels==3,0,labels)
    padded_outputs = tf.cast(logits, tf.int32)
    padded_labels = tf.cast(labels, tf.int32)

    weights_outputs = tf.logical_and(tf.not_equal(padded_outputs, 0), tf.not_equal(padded_outputs, 1))
    weights_labels = tf.logical_and(tf.not_equal(padded_labels, 0), tf.not_equal(padded_labels, 1))
    
    true_p = tf.logical_and(isin(padded_labels, padded_outputs), weights_outputs)
    out_in_lab = tf.logical_or(isin(padded_labels, padded_outputs), tf.logical_not(weights_outputs))
    lab_in_out = tf.logical_or(isin(padded_labels, padded_outputs), tf.logical_not(weights_labels))
    
    true_positives = tf.cast(true_p, tf.float32)
    false_positives = tf.cast(tf.logical_not(out_in_lab), tf.float32)
    false_negatives = tf.cast(tf.logical_not(lab_in_out), tf.float32)

    return tf.reduce_sum(true_positives), tf.reduce_sum(false_positives), tf.reduce_sum(false_negatives)


def my_metrics(logits, labels):

    true_positives, false_positives, false_negatives = true_false_positives(logits, labels)

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f_measure = 2 * precision * recall / (precision + recall)

    metrics = (tf.keras.metrics.Mean)

    return precision, recall, f_measure


In [32]:
true_false_positives(np.array([test[0][1]]),np.array([a[1][1]]))

NameError: name 'test' is not defined

In [35]:
z = my_metrics(test[0],a[1])

In [40]:
z[1].item()

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'item'

In [43]:
a = 5 + float(z[1])
a

5.931034505367279

# Restes

In [ ]:
test_processor = preprocessing.TextVectorization(
    standardize=None,
    max_tokens=None)

data_test = pd.read_csv("data/extrait500_corpus_test_code_target.csv", encoding='utf-8', sep=";")
test_processor.adapt(data_test.target)

In [ ]:
vocabulaire_test = test_processor.get_vocabulary()
for i in range(len(vocabulaire_test)):
    vocabulaire_test[i] = [vocabulaire_test[i]]

In [ ]:
import csv

with open("data/vocabulaire_test.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(vocabulaire_test)

In [ ]:
def preprocess1(features):


    txt = input_text_processor(features.pop('Dico'))
    
    return txt


def input_fn1(dataset_name, batch_size, preprocess = preprocess):

    data = tf.data.experimental.make_csv_dataset(
        path_data+'%s.csv' % dataset_name,
        batch_size=batch_size,
        column_names=['Dico'],
        column_defaults=['a'],
        header=True,
        shuffle=False,
        shuffle_buffer_size=20000,
        sloppy=True,
        prefetch_buffer_size=1,
        use_quote_delim=True,
        field_delim=';'

    )

    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.prefetch(buffer_size=1)

    return data

In [ ]:
dataset1 = input_fn1(dataset_name = "vocabulaire_test",batch_size =31, preprocess  = preprocess1
                  )
iterator = iter(dataset1)
iterator

In [ ]:
test2 = Model_test.predict(b)


In [ ]:
for i in range(len(test2[0])):
    vocab = target_processor.get_vocabulary()
    print(i)
    print(" ".join([vocab[each] for each in tf.squeeze(test[0][i])])
)

In [ ]:
target_processor.get_vocabulary()
